In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# df = pd.read_csv('../data/ufo_clean.csv', low_memory=False)
df = pd.read_csv('../data/processed/ufos_processed.csv', low_memory=False, on_bad_lines='skip')

In [4]:
# Setting up for DBSCAN clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

# DBSCAN clustering on the data
# Standardize the data to have a mean of ~0 and a variance of 1
X = StandardScaler().fit_transform(df) # Standardize features by removing the mean and scaling to unit variance
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
# Get the labels
labels = db.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# Number of noise points in the full set.
n_noise_ = list(labels).count(-1)
# Print the results
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, labels))
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# Plot the results
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
            for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
            markeredgecolor='k', markersize=14) # Plot the core samples

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
            markeredgecolor='k', markersize=6) # Plot the outliers

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show();


ModuleNotFoundError: No module named 'sklearn.datasets.samples_generator'